<a href="https://colab.research.google.com/github/secutron/ShowTime/blob/master/DH_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code setup: Imports & methods

Get the classes and models

In [0]:
!git clone https://github.com/26medias/keras-face-toolbox.git
!mv keras-face-toolbox/models models
!mv keras-face-toolbox/utils utils
!rm -r keras-face-toolbox
!gdown https://drive.google.com/uc?id=1H37LER8mRRI4q_nxpS3uQz3DcGHkTrNU
!mv lresnet100e_ir_keras.h5 models/verifier/insightface/lresnet100e_ir_keras.h5

Cloning into 'keras-face-toolbox'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 76 (delta 17), reused 72 (delta 13), pack-reused 0
Unpacking objects: 100% (76/76), done.
Checking out files: 100% (37/37), done.
Downloading...
From: https://drive.google.com/uc?id=1H37LER8mRRI4q_nxpS3uQz3DcGHkTrNU
To: /content/lresnet100e_ir_keras.h5
262MB [00:01, 192MB/s]


Pip install

In [0]:
!rm -r Faces/*

rm: cannot remove 'Faces/*': No such file or directory


In [0]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip show keras-vggface
!pip install matplotlib
!pip install mtcnn
!pip install bs4
!pip install selenium

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-crxp3wx1
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-crxp3wx1
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=81ba0f3bd8a0f24ab389d003b577a91d5433b48710e3fd2745e7a2d456205393
  Stored in directory: /tmp/pip-ephem-wheel-cache-hjrzc92f/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: keras, six, scipy, pillow, h5py, pyyaml, numpy
Required-by: 
     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=be9f1f22569243f41dca8

### Code

#### Directory Structure

- Videos
  - [video_filename]
- Faces
  - [group_id]
    - Faces
    - Embeddings
    - Landmarks
    - Segmentations
    - Previews

In [0]:
from IPython.display import HTML, display
import time
import requests
import ntpath
import cv2
import math
import os, sys
from matplotlib import pyplot
from PIL import Image
import numpy as np
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import glob
import mtcnn
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from bs4 import BeautifulSoup
from selenium import webdriver
import csv
from models.detector import face_detector
from models.parser import face_parser
from utils.visualize import show_parsing_with_annos

# Create the detector, using default weights
print("Creating the detector model")
detector = MTCNN()

# Create a vggface model
print("Creating the face embedding model")
embedding_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Create a face detector
print("Creating the face detector model")
fd = face_detector.FaceAlignmentDetector(lmd_weights_path="models/detector/FAN/2DFAN-4_keras.h5")

# Create a face parser (segmentation)
print("Creating the face segmentation model")
prs = face_parser.FaceParser()


Using TensorFlow backend.
W0904 04:04:54.808999 140485065951104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/mtcnn.py:187: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0904 04:04:54.844135 140485065951104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/mtcnn.py:193: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



landmark_detector FILE_PATH /content/models/detector
face_detector FILE_PATH /content/models/detector
Creating the detector model


W0904 04:04:56.106729 140485065951104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/network.py:43: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0904 04:04:56.108098 140485065951104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/layer_factory.py:88: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0904 04:04:56.115648 140485065951104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/layer_factory.py:79: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0904 04:04:56.117411 140485065951104 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance 

Creating the face embedding model


W0904 04:05:06.162803 140485065951104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4271: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



94699520/94694792 [==============================] - 1s 0us/step
Creating the face detector model
Creating the face segmentation model


W0904 04:05:14.120714 140485065951104 deprecation_wrapper.py:119] From /content/models/parser/BiSeNet/bisenet.py:56: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0904 04:05:14.783345 140485065951104 deprecation_wrapper.py:119] From /content/models/parser/BiSeNet/bisenet.py:58: The name tf.image.resize_bilinear is deprecated. Please use tf.compat.v1.image.resize_bilinear instead.



In [0]:

# The variables
DIR_VIDEOS = "Videos"
DIR_FACES = "Faces"
CAPTURE_FPS  = 23 # We'll extract 1 images per second of video

if not os.path.isdir(DIR_VIDEOS):
  os.mkdir(DIR_VIDEOS, 755);
if not os.path.isdir(DIR_FACES):
  os.mkdir(DIR_FACES, 755);


# Quick test mode
TEST_MODE = False

if TEST_MODE is True:
  CAPTURE_FPS = 23

# The methods
# ===========

# Colab progress bar
def progress(value, max=100):
    return HTML('<progress value="{value}" max="{max}" style="width: 50%"> {value}</progress>'.format(value=value, max=max))

# Convert a value from one range to another
def rangeConvert(x, in_min, in_max, out_min, out_max):
  return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min

# Get the directory of a filename
def getDir(filename):
  p = Path(filename);
  return p.parts[len(p.parts)-2]

# Dowload a video from a url
def downloadFile(url):
  print("Downloading ", url)
  filename = DIR_VIDEOS+"/"+ntpath.basename(url)
  if os.path.exists(filename):
    return filename
  myfile = requests.get(url)
  open(filename, 'wb').write(myfile.content)
  print(filename," downloaded.")
  return filename

# Resize an image
def resize_image(im, max_size=768):
    if np.max(im.shape) > max_size:
        ratio = max_size / np.max(im.shape)
        print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
        return cv2.resize(im, (0,0), fx=ratio, fy=ratio)
    return im


def imageFilesToGrid(directory, outputFilename):
  filenames = glob.glob(directory+'/*.jpg')
  print(directory, ": ", len(filenames), " images")
  if len(filenames) < 4:
    return False
  result_figsize_resolution = 10 # 1 = 100px
  
  images_count = len(filenames)
  # Calculate the grid size:
  grid_size = math.ceil(math.sqrt(images_count))
  
  # Create plt plot:
  fig, axes = pyplot.subplots(grid_size, grid_size, figsize=(result_figsize_resolution, result_figsize_resolution))
  
  current_file_number = 0
  for image_filename in filenames:
      x_position = current_file_number % grid_size
      y_position = current_file_number // grid_size
      plt_image = pyplot.imread(image_filename)
      axes[x_position, y_position].imshow(plt_image)
      current_file_number += 1
  pyplot.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=1.0)
  pyplot.savefig(outputFilename)
  #pyplot.show()

def exportImageGrids(directory, outputDirectory):
  print("Exporting image grids...")
  dirs = os.listdir(directory)
  dirs.sort()
  ndirs = len(dirs)
  for n,dir in enumerate(dirs):
    if dir is not "ALL":
      imageFilesToGrid(directory+"/"+dir, outputDirectory+"/"+dir+".jpg");
    progress(n, ndirs)

# Extract the faces from an image, return an array of numpy faces
def extractFacesFromImage(pixels, required_size=(224, 224), limit=50):
  results = detector.detect_faces(pixels)
  faces = []
  errors = 0
  for i,faceData in enumerate(results):
    if len(faces) > limit:
      break
    x1, y1, width, height = faceData['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    try:
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = asarray(image)
      faces.append(face_array)
      if limit==1:
        return face_array
    except:
      errors+=1
  if limit==1 and len(faces)==0:
    return False
  return faces;

# Extract the faces from an image, return an array of numpy faces & landmarks
def extractFacesAndLandmarksFromImage(pixels, required_size=(224, 224), limit=50):
  rw, rh = required_size
  results, landmarks = fd.detect_face(pixels, with_landmarks=True)
  nResults = len(results)
  faces = []
  errors = 0
  for i,bbox in enumerate(results):
    if len(faces) > limit:
      break
    # Get the face
    x0, y0, x1, y1, score = bbox
    # Find the center of the face
    w       = x1-x0
    h       = y1-y0
    xCenter = x0+int(w/2)
    yCenter = y0+int(h/2)
    if w>h:
      y0 = yCenter-int(w/2)
      y1 = yCenter+int(w/2)
    if h>w:
      x0 = xCenter-int(h/2)
      x1 = xCenter+int(h/2)
    x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
    face = pixels[x0:x1, y0:y1, :]
    # Recalculate the landmarks coordinates
    for li in range(len(landmarks[i])): 
      landmark = landmarks[i][li]
      lx, ly = landmark
      landmarks[i][li] = (rangeConvert(lx-x0, 0, face.shape[1], 0, rw), rangeConvert(ly-y0, 0, face.shape[0], 0, rh))
    # Resize pixels to the model size
    try:
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = asarray(image)
      faces.append(face_array)
      if limit==1:
        return face_array
    except:
      errors+=1
  if limit==1 and len(faces)==0:
    return False
  return faces, landmarks

# Extract the faces from an image, return an array of numpy faces & landmarks
def extractFacesLandmarksAndSegmentationFromImage(pixels, required_size=(224, 224), limit=50):
  rw, rh = required_size
  results, landmarks = fd.detect_face(pixels, with_landmarks=True)
  nResults = len(results)
  faces = []
  segmentations = []
  errors = 0
  for i,bbox in enumerate(results):
    if len(faces) > limit:
      break
    # Get the face
    x0, y0, x1, y1, score = bbox
    # Find the center of the face
    w       = x1-x0
    h       = y1-y0
    xCenter = x0+int(w/2)
    yCenter = y0+int(h/2)
    if w>h:
      y0 = yCenter-int(w/2)
      y1 = yCenter+int(w/2)
    if h>w:
      x0 = xCenter-int(h/2)
      x1 = xCenter+int(h/2)
    x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
    face = pixels[x0:x1, y0:y1, :]
    # Recalculate the landmarks coordinates
    for li in range(len(landmarks[i])): 
      landmark = landmarks[i][li]
      lx, ly = landmark
      landmarks[i][li] = (rangeConvert(lx-x0, 0, face.shape[1], 0, rw), rangeConvert(ly-y0, 0, face.shape[0], 0, rh))
    # Resize pixels to the model size
    try:
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = asarray(image)
      faces.append(face_array)
      # Get the segmentation on the resized image
      segmentation = prs.parse_face(face_array)
      segmentations.append(segmentation)
      if limit==1:
        return face_array
    except:
      errors+=1
  if limit==1 and len(faces)==0:
    return False
  return faces, landmarks, segmentations

# Export the frames out of a video at a specific fps
def videoToFaces(filename, skipFrame=10, maxFrame=0):
  print("Extracting faces from the video frames...")
  basename = os.path.splitext(ntpath.basename(filename))[0]
  #print("basename:", basename)
  cap = cv2.VideoCapture(filename)
  # Get the video's FPS
  fps = cap.get(cv2.CAP_PROP_FPS)
  nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  if TEST_MODE is True:
    maxFrame = int(nframes/4)
  processFrames = int(nframes/skipFrame)
  print(basename, ": fps: ",fps, "skipFrame:",skipFrame," Frames: ", str(processFrames)+"/"+str(nframes))
  out = display(progress(0, processFrames), display_id=True)
  i = 0
  c = 0
  faces = []
  landmarks = []
  segmentations = []
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      i+=1
      if maxFrame>0 and i > maxFrame:
        break;
      #print(i, "-", i % skipFrame)
      if (i % skipFrame == 0):
        c+=1
        #print("Checking faces in frame #"+str(i))
        #frameFaces = extractFacesFromImage(frame)
        frameFaces, frameLandmarks, frameSegmentations = extractFacesLandmarksAndSegmentationFromImage(frame)
        out.update(progress(c, processFrames))
        for nf, f in enumerate(frameFaces):
          faces.append(f)
          landmarks.append(frameLandmarks[nf])
          segmentations.append(frameSegmentations[nf])
      else:
        continue
      #cv2.imwrite(DIR_IMAGES+"/"+basename+'/'+str(round((i-1)/fps,2))+'sec.jpg',frame)
  cap.release()
  cv2.destroyAllWindows()
  print(basename, " processed.")
  print(processFrames,"/",nframes," frames analyzed.")
  print(len(faces), " faces found.")
  return faces, landmarks, segmentations


# Show a few images
def showImages(images, width=4):
  fig = pyplot.figure(figsize=(width, math.ceil(len(images)/width)))
  for i in range(len(images)):
      pyplot.subplot(width, math.ceil(len(images)/width), i+1)
      pyplot.imshow(images[i])
      pyplot.axis('off')
  pyplot.savefig('preview.png')
  pyplot.show()

# Save an array of images to files
def saveImages(images, dest, names=False, prefix="", showProgress=True):
  if not os.path.isdir(dest):
    os.mkdir(dest, 755);
  nImages = len(images)
  if showProgress is True:
    print("Saving ",nImages," images to ", dest)
    out = display(progress(0, nImages), display_id=True)
  filenames = []
  for n, image in enumerate(images):
    if names is False:
      filename = dest+"/"+prefix+('{:04d}'.format(n))+'.jpg'
    else:
      filename = dest+"/"+prefix+str(names[n])+'.jpg'
    cv2.imwrite(filename, image)
    filenames.append(filename)
    if showProgress is True:
      out.update(progress(n, nImages))
  return filenames

# Save Numpy Arrays to files
def saveNpArrays(npArrays, dest, names=False, prefix="", showProgress=True):
  if not os.path.isdir(dest):
    os.mkdir(dest, 755);
  nArrays = len(npArrays)
  if showProgress is True:
    print("Saving ",nArrays," numpy arrays to ", dest)
    out = display(progress(0, nArrays), display_id=True)
  filenames = []
  for n, npArray in enumerate(npArrays):
    if names is False:
      filename = dest+"/"+prefix+('{:04d}'.format(n))+'.npy'
    else:
      filename = dest+"/"+prefix+str(names[n])+'.npy'
    np.save(filename, npArray)
    filenames.append(filename)
    if showProgress is True:
      out.update(progress(n, nArrays))
  return filenames


# Extract faces and calculate face embeddings for a list of photo files
def get_embeddings(faces):
  print("Calculating the embeddings...")
	# convert into an array of samples
  samples = asarray(faces, 'float32')
  # prepare the face for the model, e.g. center pixels
  samples = preprocess_input(samples, version=2)
  # perform prediction
  embeddings = embedding_model.predict(samples)
  return embeddings


# Determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, threshold=0.5):
  # calculate distance between embeddings
  score = cosine(known_embedding, candidate_embedding)
  return score >= threshold

# Cluster the faces by cosine distance
def clusterFaces(faces, embeddings, landmarks, segmentations):
  groups = [] # Array of dict {faces:[], embeddings: []}
  nFaces = len(faces)
  print("Clustering ",nFaces," faces...")
  out = display(progress(0, nFaces), display_id=True)
  # For each faces
  for n, face in enumerate(faces):
    out.update(progress(n, nFaces))
    if len(groups)==0:
      groups.append({
        "faces":         [face],
        "names":         [n],
        "embeddings":    [embeddings[n]],
        "landmarks":     [landmarks[n]],
        "segmentations": [segmentations[n]]
      })
    else:
      # Not the first face, match it against all the groups, see if the average of cosine distance match an existing face
      scores = [] # array of dict {group: n, embeddings: []}
      for g, group in enumerate(groups):
        groupScores = []
        for embedding in group["embeddings"]:
          groupScores.append(cosine(embedding, embeddings[n]))
        score = np.mean(groupScores)
        scores.append({
            "group": g,
            "score": score
        })
      # Sort the scores for each group by lowest score, check if that score is below the threshold
      scores = sorted(scores, key = lambda i: i["score"], reverse=False)
      if scores[0]["score"] <= 0.5:
        # Add to the existing group the face matches
        groups[scores[0]["group"]]["landmarks"].append(landmarks[n])
        groups[scores[0]["group"]]["embeddings"].append(embeddings[n])
        groups[scores[0]["group"]]["segmentations"].append(segmentations[n])
        groups[scores[0]["group"]]["faces"].append(face)
        groups[scores[0]["group"]]["names"].append(n)
        #print("[Matched] face #", n, " to group #", scores[0]["group"], "score:", scores[0]["score"])
      else:
        groups.append({
          "faces":       [face],
          "names":       [n],
          "embeddings":  [embeddings[n]],
          "landmarks":   [landmarks[n]],
        "segmentations": [segmentations[n]]
        })
        #print("[New face] face #", n, " / Best score:", scores[0]["score"])
  return groups;

# Cluster all the faces from a remote video
def clusterFacesOnVideo(url):
  print("Processing ", url);
  # Download the video
  videoFilename = downloadFile(url)
  
  # Get the directories name for that video
  # /Faces/[dirname]/Faces
  # /Faces/[dirname]/Embeddings
  # /Faces/[dirname]/Landmarks
  # /Faces/[dirname]/Segmentations
  # /Faces/[dirname]/Previews
  dirname          = os.path.splitext(ntpath.basename(videoFilename))[0]
  
  dirClustered     = DIR_FACES+"/"+dirname
  dirFaces         = dirClustered+"/Faces/"
  dirEmbeddings    = dirClustered+"/Embeddings/"
  dirLandmarks     = dirClustered+"/Landmarks/"
  dirSegmentations = dirClustered+"/Segmentations/"
  dirPreviews      = dirClustered+"/Previews/"
  
  if os.path.exists(dirPreviews):
    # Video already processed, go to the next one
    print("Video already processed.")
    #return False
  
  # Create the directories
  if not os.path.isdir(dirClustered):
    os.mkdir(dirClustered, 755);
  if not os.path.isdir(dirFaces):
    os.mkdir(dirFaces, 755);
  if not os.path.isdir(dirEmbeddings):
    os.mkdir(dirEmbeddings, 755);
  if not os.path.isdir(dirLandmarks):
    os.mkdir(dirLandmarks, 755);
  if not os.path.isdir(dirSegmentations):
    os.mkdir(dirSegmentations, 755);
  if not os.path.isdir(dirPreviews):
    os.mkdir(dirPreviews, 755);
  
  # Open a CSV to save the datasets
  with open(dirClustered+"/"+dirname+".csv", "w") as csvfile:
    fieldnames = ["video_name", "face_group", "image_filename", "embeddings_filename", "landmarks_filename"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    # Find the faces on the video
    faces, landmarks, segmentations  = videoToFaces(videoFilename, CAPTURE_FPS)
    nFaces       = len(faces)
    print(nFaces," faces detected")

    # Get the embedding for all the faces
    embeddings    = get_embeddings(faces)
    
    # Cluster the faces using cosine distance
    clusters      = clusterFaces(faces, embeddings, landmarks, segmentations)
    nClusters     = len(clusters)

    # Export each face group
    print("Saving ",nClusters," face clusters...")
    for n, group in enumerate(clusters):
      
      ngImg = len(group["faces"])
      ngEbd = len(group["embeddings"])
      ngldk = len(group["landmarks"])
      
      # Save the face as an image
      image_filenames      = saveImages(group["faces"], dirFaces+"/"+('{:04d}'.format(n)), showProgress=False)
      
      # Save the embedding as a numpy array
      embeddings_filenames = saveNpArrays(group["embeddings"], dirEmbeddings+"/"+('{:04d}'.format(n)), showProgress=False)
      
      # Save the landmarks as a numpy array
      landmarks_filenames  = saveNpArrays(group["landmarks"], dirLandmarks+"/"+('{:04d}'.format(n)), showProgress=False)
      
      # Save the segmentations as a numpy array
      image_filenames      = saveNpArrays(group["segmentations"], dirSegmentations+"/"+('{:04d}'.format(n)), showProgress=False)
      
      
      # Update the CSV
      for i, image_filename in enumerate(image_filenames):
        writer.writerow({
            "video_name": dirname,
            "face_group": n,
            "image_filename": image_filename,
            "embeddings_filename": embeddings_filenames[i],
            "landmarks_filename": landmarks_filenames[i]
        })


    # Build grids to show each face groups
    exportImageGrids(dirFaces, dirPreviews)


def clusterFacesFromVideos(urls):
  nUrls = len(urls)
  for n,url in enumerate(urls):
    clusterFacesOnVideo(url)

def fetchAllHDVideos(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html5lib")
  links = soup.find_all('a')
  videos = []
  for tag in links:
    link = tag.get('href', None)
    if link is not None and 'h480p' in link:
      videos.append(link)
  return videos

## Execute on all the videos

This is going to take a few days...


In [0]:

# Fetch all the HD trailers by webscrapping the webpage
vids = fetchAllHDVideos("https://www.davestrailerpage.co.uk/")
#vids = vids[:1] # Limit to the 1st video

# Cluster the faces from a bunch of videos
clusterFacesFromVideos(vids)

# Save the faces
!tar -zcf faces.tar.gz Faces

# Upload to Cloud Storage
!gcloud config set project deep-learning-files
!gsutil cp  ./faces.tar.gz gs://tf-face-angle-translation/datasets/faces-clustered-very-large.tar.gz


Processing  http://trailers.apple.com/movies/paramount/terminator-dark-fate/terminator-dark-fate-trailer-2_h480p.mov
Video already processed.
Extracting faces from the video frames...
terminator-dark-fate-trailer-2_h480p : fps:  23.976023976023978 skipFrame: 23  Frames:  164/3774


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars


terminator-dark-fate-trailer-2_h480p  processed.
164 / 3774  frames analyzed.
111  faces found.
111  faces detected
Calculating the embeddings...
Clustering  111  faces...


Saving  26  face clusters...
Exporting image grids...
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0000 :  27  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0001 :  10  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0002 :  3  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0003 :  6  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0004 :  2  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0005 :  12  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0006 :  2  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0007 :  2  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0008 :  1  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0009 :  1  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0010 :  2  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0011 :  5  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0012 :  10  images
Faces/terminator-dark-fate-trailer-2_h480p/Faces//0013 :  